# 데이터 준비

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# 데이터 전처리

In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.groupby('movie_id')['movie_id'].head()

0         1193
1          661
2          914
3         3408
4         2355
          ... 
996177    2192
996467    3245
996637    3336
997181      59
997451    3443
Name: movie_id, Length: 17223, dtype: int64

#### ratings와 movies 데이터 조인

In [7]:
data = pd.merge(ratings, movies, how='inner', on = 'movie_id')

In [8]:
data

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


---

# 데이터 탐색

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)


#### 데이터셋 전체 크기

In [9]:
len(data)

836478

#### ratings에 있는 유니크한 영화 개수

In [10]:
data['movie_id'].nunique()

3628

#### ratings에 있는 유니크한 사용자 수

In [11]:
data['user_id'].nunique()

6039

#### 가장 인기 있는 영화 30개(인기순)

In [12]:
movie_count = data.groupby(['movie_id', 'title'])['user_id'].count()
movie_count.sort_values(ascending=False).head(60)

movie_id  title                                                
2858      American Beauty (1999)                                   3211
260       Star Wars: Episode IV - A New Hope (1977)                2910
1196      Star Wars: Episode V - The Empire Strikes Back (1980)    2885
1210      Star Wars: Episode VI - Return of the Jedi (1983)        2716
2028      Saving Private Ryan (1998)                               2561
589       Terminator 2: Judgment Day (1991)                        2509
593       Silence of the Lambs, The (1991)                         2498
1198      Raiders of the Lost Ark (1981)                           2473
1270      Back to the Future (1985)                                2460
2571      Matrix, The (1999)                                       2434
480       Jurassic Park (1993)                                     2413
2762      Sixth Sense, The (1999)                                  2385
608       Fargo (1996)                                             2371


#### 내가 선호하는 영화를 5가지 골라서 데이터에 추가

In [13]:
import calendar
import time

ts = calendar.timegm(time.gmtime())

my_movie_ids = [2571, 1580, 1270, 3114, 1]
jiyeon_user_id = data.user_id.max() + 1

my_movies = []
for movie_id in my_movie_ids:
    m = movies[movies['movie_id'] == movie_id]
    my_movies.append({
        'user_id': jiyeon_user_id, 
        'movie_id': movie_id, 
        'counts': 7, 
        'timestamp': ts, 
        'title': m['title'].values[0], 
        'genre': m['genre'].values[0]
    })
    
my_favorite = pd.DataFrame(my_movies, columns=data.columns)

In [14]:
my_favorite

,user_id,movie_id,counts,timestamp,title,genre
0,6041,2571,7,1645776414,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1,6041,1580,7,1645776414,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2,6041,1270,7,1645776414,Back to the Future (1985),Comedy|Sci-Fi
3,6041,3114,7,1645776414,Toy Story 2 (1999),Animation|Children's|Comedy
4,6041,1,7,1645776414,Toy Story (1995),Animation|Children's|Comedy


In [15]:
if not data.isin({'user_id':[jiyeon_user_id]})['user_id'].any():
    data = data.append(my_favorite)

data.tail(10)

,user_id,movie_id,counts,timestamp,title,genre
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,1016563709,Identification of a Woman (Identificazione di ...,Drama
0,6041,2571,7,1645776414,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1,6041,1580,7,1645776414,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2,6041,1270,7,1645776414,Back to the Future (1985),Comedy|Sci-Fi
3,6041,3114,7,1645776414,Toy Story 2 (1999),Animation|Children's|Comedy
4,6041,1,7,1645776414,Toy Story (1995),Animation|Children's|Comedy


#### 데이터가 잘 들어갔는지 확인

In [16]:
data[data['user_id'] == jiyeon_user_id]

,user_id,movie_id,counts,timestamp,title,genre
0,6041,2571,7,1645776414,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1,6041,1580,7,1645776414,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2,6041,1270,7,1645776414,Back to the Future (1985),Comedy|Sci-Fi
3,6041,3114,7,1645776414,Toy Story 2 (1999),Animation|Children's|Comedy
4,6041,1,7,1645776414,Toy Story (1995),Animation|Children's|Comedy


#### 유저, 영화 indexing 하는 코드

- user_id 기준으로 유저 인덱싱
- title 기준으로 영화 인덱싱

In [17]:
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [18]:
print(user_to_idx[jiyeon_user_id]) # 6040명 유저에서 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(movie_to_idx['Matrix, The (1999)'])

6039
124


#### 추가된 데이터도 인덱싱하기

In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,timestamp,title,genre
0,0,1193,5,978300760,0,Drama
1,1,1193,5,978298413,0,Drama
2,2,1193,4,978220179,0,Drama
3,3,1193,4,978199279,0,Drama
4,4,1193,5,978158471,0,Drama
...,...,...,...,...,...,...
0,6039,2571,7,1645776414,124,Action|Sci-Fi|Thriller
1,6039,1580,7,1645776414,175,Action|Adventure|Comedy|Sci-Fi
2,6039,1270,7,1645776414,22,Comedy|Sci-Fi
3,6039,3114,7,1645776414,50,Animation|Children's|Comedy


# CSR matrix 만들기

- 기존 user_id, movie_id로는 만들면 에러를 발생했습니다. -> CSR matrix에 넣는 데이터는 순서대로 데이터 크기만큼의 index 순번이어야 함
    - 새로 인덱싱한 컬럼을 기준으로 csr matrix를 만듬
- 총 6040명인 user 기준으로 CSR matrix가 구성됐습니다.

In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# MF 모델 학습하기

### AlternatingLeastSquares 클래스
- factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU 사용할 것인지
- iterations : epochs와 같은 의미 (데이터를 몇 번 반복해서 학습할 것인지

In [21]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

csr_data_transpose = csr_data.T
csr_data_transpose

als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

---

# 선호도 파악하기

> 내가 선호하는 5가지 영화 중 1개 & 그 외의 영화 1개 -> 훈련된 모델이 예측한 나의 선호도를 파악하기

### 1) 내가 원래 선호하던 5가지 영화 중 1개 
- Man In Black 영화 선호도를 확인해보기

In [23]:
jiyeon, man_in_black = user_to_idx[jiyeon_user_id], movie_to_idx['Men in Black (1997)']
jiyeon_vector, man_in_black_vector = als_model.user_factors[jiyeon], als_model.item_factors[man_in_black]

In [24]:
jiyeon_vector

array([-0.23758286,  0.30074987,  0.17326394,  0.563944  ,  1.2126328 ,
        0.2317357 , -0.9363153 ,  1.0189747 ,  0.6819774 ,  0.45132038,
       -0.2094896 ,  0.850293  ,  0.21444596,  0.03765312, -0.40517867,
       -0.03254031,  0.7294786 , -0.55818474, -0.35695496,  0.15268695,
       -0.0845459 , -0.01293401, -0.2671625 , -0.06574966,  0.5136933 ,
       -0.84315765,  0.43996996, -1.3515253 , -0.01724607,  0.18981668,
       -0.34490874,  0.16955782,  0.10552771, -0.76932234, -0.6555545 ,
        0.6910811 , -0.90843046, -0.06507647,  0.5806643 , -1.0342112 ,
       -0.8525131 , -0.6069823 , -0.38871273,  0.53127086, -1.0696485 ,
       -0.12141237, -0.59144855, -0.38313097,  0.37234303,  0.42941728,
        0.4718578 , -1.4794798 , -1.222309  , -0.28669265,  0.623441  ,
       -0.2763535 , -0.23059084,  0.58885866, -0.3722807 ,  0.3235594 ,
       -0.49971738,  0.07009993,  0.34157357, -0.5999077 , -0.48253328,
       -0.13803825, -0.13084047, -0.9924836 ,  0.5654055 ,  0.75

In [25]:
man_in_black_vector

array([-8.5443738e-05,  3.1069664e-02,  4.1186693e-03,  5.7890597e-03,
        3.4684349e-02,  2.3954093e-02, -8.1466204e-03, -3.8924413e-03,
        3.4683224e-02,  2.3674327e-03, -1.0376813e-02,  1.1118839e-02,
        9.7939484e-03,  2.3995033e-02, -3.9803952e-02,  3.4048550e-02,
        2.1988647e-02,  1.5243468e-03,  7.2227116e-03,  1.5640680e-02,
        2.5555342e-02,  8.9092785e-03, -2.1460666e-03,  5.0160207e-04,
        1.7369157e-02, -1.6221231e-02,  1.8496460e-03, -4.2060632e-02,
        6.2914053e-03, -6.8564564e-03, -1.7069589e-03,  3.7438668e-02,
        8.7266183e-03, -3.8028065e-02, -1.0987275e-02,  9.4029205e-03,
        9.7004669e-03,  6.0150307e-03,  4.4230990e-02, -1.4961043e-02,
        3.1075554e-03,  1.8391175e-02,  1.4346210e-02,  6.0959198e-03,
       -1.9844178e-02, -1.9995451e-02,  5.9113484e-03,  3.6454708e-03,
        4.1803710e-02,  2.1395944e-02, -7.3675048e-03, -3.2991229e-03,
       -1.1336498e-02,  1.7939399e-04,  2.0259175e-02, -1.3558588e-02,
      

In [26]:
np.dot(jiyeon_vector, man_in_black_vector)

0.5959657

### 2) 그 외 영화 1개 
- 별로 좋아하지 않은 영화(Schindler's List) 선호도를 확인해보기     

In [27]:
jiyeon, schindler = user_to_idx[jiyeon_user_id], movie_to_idx["Schindler's List (1993)"]
jiyeon_vector, schindler_vector = als_model.user_factors[jiyeon], als_model.item_factors[schindler]

In [28]:
np.dot(jiyeon_vector, schindler_vector)

0.07323523

#### [원래 선호하던 영화]가 [그 외 영화] 보다 선호도가 더 높은 것을 확인할 수 있습니다. 실제 선호도에 따른 수치가 적절하게 나왔습니다.

---

> 추천을 진행하기 전에 미리 분석한 결과를 공유하면
>
> 별점으로 학습하여 추천한 영화들이 **genre(장르)와 강한 연관성**이 있음을 확인할 수 있었습니다.
>
> 따라서 장르 데이터와 함께 추천 결과를 비교해보겠습니다.


# 내가 좋아하는 영화와 비슷한 영화를 추천받기

#### 설정한 좋아하는 영화 리스트

In [54]:
my_movielist = ('Matrix, The (1999)', 'Men in Black (1997)', 'Back to the Future (1985)', 'Toy Story 2 (1999)', 'Toy Story (1995)')

movies.loc[movies['title'].isin(my_movielist)]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1250,1270,Back to the Future (1985),Comedy|Sci-Fi
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy


#### 제가 좋아하는 영화 장르를 살펴보면,
``중복된 장르 >= 2``를 기준으로 

- Comedy(4)
- SF(3)
- Animation(2), Children's(2), Action(2) 

임을 알 수 있습니다.

### 이를 기준으로 비슷한 영화를 추천 받아 보겠습니다.

In [31]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

In [32]:
def get_similar_movie(movie: str, num=10):
    movie_id = movie_to_idx[movie]
    similar_movie = als_model.similar_items(movie_id, N=num)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

### 매트릭스와 비슷한 영화들

- Matrix, The (1999) : Action|Sci-Fi|Thriller

In [55]:
similar_movies = get_similar_movie('Matrix, The (1999)', 15)
movies.loc[movies['title'].isin(similar_movies)]

,movie_id,title,genre
31,32,Twelve Monkeys (1995),Drama|Sci-Fi
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
453,457,"Fugitive, The (1993)",Action|Thriller
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1220,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
1491,1527,"Fifth Element, The (1997)",Action|Sci-Fi
1533,1573,Face/Off (1997),Action|Sci-Fi|Thriller
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
1568,1610,"Hunt for Red October, The (1990)",Action|Thriller


**매트릭스(Action|Sci-Fi|Thriller)와 비슷한 장르의 영화들을 추천받았습니다.**

### 토이스토리와 비슷한 영화들

- Toy Story 2 (1999) : Animation|Children's|Comedy

In [56]:
similar_movies = get_similar_movie('Toy Story 2 (1999)', 15)
movies.loc[movies['title'].isin(similar_movies)]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
33,34,Babe (1995),Children's|Comedy|Drama
360,364,"Lion King, The (1994)",Animation|Children's|Musical
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
591,595,Beauty and the Beast (1991),Animation|Children's|Musical
1526,1566,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
2315,2384,Babe: Pig in the City (1998),Children's|Comedy
2327,2396,Shakespeare in Love (1998),Comedy|Romance
2618,2687,Tarzan (1999),Animation|Children's


**Toy Story 2 (1999)(Animation|Children's|Comedy)와 비슷한 장르의 영화들을 추천받았습니다.**

---

# 내가 가장 좋아할 만한 영화들을 추천받기

In [35]:
user = user_to_idx[jiyeon_user_id]

In [36]:
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 0.5762428),
 (107, 0.54209596),
 (4, 0.5114997),
 (322, 0.41800052),
 (62, 0.39809644),
 (160, 0.38462758),
 (38, 0.3334483),
 (110, 0.3208392),
 (670, 0.31960294),
 (317, 0.30118093),
 (26, 0.29525116),
 (141, 0.28703824),
 (48, 0.28647113),
 (33, 0.28223056),
 (474, 0.2719693),
 (200, 0.27159557),
 (479, 0.27029067),
 (5, 0.26124462),
 (243, 0.25994566),
 (372, 0.2594849)]

#### 내가 좋아할만한 영화들을 추천받아봅니다.

In [37]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Jurassic Park (1993)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Total Recall (1990)',
 'Forrest Gump (1994)',
 'Sixth Sense, The (1999)',
 'Groundhog Day (1993)',
 'Galaxy Quest (1999)',
 'Twelve Monkeys (1995)',
 'E.T. the Extra-Terrestrial (1982)',
 'Fugitive, The (1993)',
 'Saving Private Ryan (1998)',
 'Aladdin (1992)',
 'Chicken Run (2000)',
 'Terminator, The (1984)',
 'Contact (1997)',
 'Princess Bride, The (1987)',
 'Ghostbusters (1984)',
 'X-Men (2000)']

#### 이 중에서 원래 좋아하던 영화("Aladdin (1992)")를 골라 추천 기여도를 확인해봅니다.

In [38]:
aladdin = movie_to_idx['Aladdin (1992)']
explain = als_model.explain(user, csr_data, itemid=aladdin)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.1758038457138342),
 ('Toy Story 2 (1999)', 0.06341676831393986),
 ('Back to the Future (1985)', 0.05129906372841753),
 ('Men in Black (1997)', 0.015620400609459097),
 ('Matrix, The (1999)', -0.027145758649772957)]

**"Aladdin (1992)" 영화는 토이스토리 영화 시리즈에 기여도가 많이 나옵니다. 애니메이션 장르로 묶여 추천된 것으로 보입니다.**

**추천된 영화 중에 정말 좋아하는 영화(Forrest Gump (1994))가 뜻밖에 포함되어 있어 기여도를 확인해봅니다.**

In [39]:
forrest_gump = movie_to_idx['Forrest Gump (1994)']
explain = als_model.explain(user, csr_data, itemid=forrest_gump)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Back to the Future (1985)', 0.14945138492202126),
 ('Toy Story (1995)', 0.09205643873690289),
 ('Men in Black (1997)', 0.07096853516934001),
 ('Toy Story 2 (1999)', 0.06571132739706201),
 ('Matrix, The (1999)', 0.004321840875410904)]

##### 사실 Forrest Gump (1994) 영화가 추천되서 뜻밖이었습니다. 

의도적으로 좋아하는 영화를 SF와 애니메이션 장르 영화만 설정해서 추천을 돌렸기 때문에

"Forrest Gump (1994)" 영화가 Back to the Future나 Toy Story에 기여도로부터 추천되었다는 점은 조금 이상했습니다.

그러나 Forrest Gump (1994)가 Back to the Future (1985), Toy Story (1995)와 어떤 연관이 있는지 데이터를 살펴보니,

In [52]:
movies.loc[movies['movie_id'].isin((1270, 1, 356))]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
352,356,Forrest Gump (1994),Comedy|Romance|War
1250,1270,Back to the Future (1985),Comedy|Sci-Fi


genre 데이터에서 같은 ``Comedy`` 장르라는 정보를 확인할 수 있었습니다. 기여도가 적절하게 선정된 것을 확인할 수 있었습니다.

---

# 회고

### 시도한 것들
    - 데이터셋의 정보를 더 잘 확인하기 위해 ratings와 movies 데이터를 join해서 처리했습니다.
    - CSR matrix를 기존 user_id, movie_id로는 만들려했더니 에러를 발생했습니다
        - 원인 : CSR matrix에 넣는 데이터는 순서대로 데이터 크기만큼의 index 순번이어야 함
        - 해결 : 새로 인덱싱한 컬럼을 기준으로 csr matrix를 만들어 에러를 해결
        
### 아쉬운 점
    - 추천 데이터에 timestamp를 활용해보면 어떨까 하는 아이디어가 떠올랐습니다. 시간이 지남에 따라 사람의 취향이 변할 수 있기 때문입니다. 따라서 동일 인물이더라도 기간 범주를 두어 다른 인물(취향)으로 분리해서 보는 방법은 어떨까 생각했습니다. (그러나 역시 아이디어 말고 실제 구현을 해봤어야...!)